In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Convolution2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D,ZeroPadding2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import model_from_json

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

import seaborn as sns
import pickle

#### Simple CNN Classifier

In [2]:
gender_model = Sequential()

gender_model.add(Convolution2D(filters = 64, kernel_size = (3, 3),
        padding = 'same',input_shape=(100,100,3), activation = 'relu'))
gender_model.add(MaxPooling2D(pool_size= (2, 2)))
gender_model.add(Convolution2D(filters = 32, kernel_size = (3, 3), 
        padding = 'same', activation = 'relu'))
gender_model.add(MaxPooling2D(pool_size= (2, 2)))
gender_model.add(Dropout(rate = 0.5))
gender_model.add(Flatten())
gender_model.add(Dense(128, activation = 'relu'))
gender_model.add(Dropout(rate = 0.25))
gender_model.add(Dense(64, activation = 'relu'))
gender_model.add(Dropout(rate = 0.5))
gender_model.add(Dense(1, activation='sigmoid'))

gender_model.compile(optimizer='adam', 
        loss='binary_crossentropy', metrics=['accuracy'])

gender_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2

In [4]:
gender_model.load_weights('history/gender_estimator.h5')

In [6]:
with open('history/history_gender.pickle', 'rb') as f:
    history_gender = pickle.load(f)

In [7]:
history_gender

{'loss': [0.374717387570964,
  0.306893359242422,
  0.28364146287130415,
  0.27078202166130266,
  0.259682549757647,
  0.24999686071929084,
  0.24129150870901542,
  0.2352526865135161,
  0.22726257178031375,
  0.22257302255950778,
  0.21660977555313907,
  0.2141926422613654,
  0.21012779339764498,
  0.20532101427049174,
  0.20264939047906255,
  0.19854043644483374,
  0.19575470624206304,
  0.19456989641764075,
  0.19083238230847477,
  0.1897405923849347,
  0.18597630219529862,
  0.1863146222135734,
  0.18520577526178175,
  0.1824648853448477,
  0.18033950411378896,
  0.17886778046486926,
  0.1755982732347549,
  0.17515531897785028,
  0.17397327541604882,
  0.1713298051294146,
  0.16918422975135772,
  0.16666541889031763,
  0.1659170431875424,
  0.1655762727324166,
  0.16337455218901348,
  0.1626489184928789,
  0.15975522489611985,
  0.15861947628967066,
  0.15976891068542506,
  0.1574238278869258,
  0.1552869480461665,
  0.15451319759486679,
  0.15457333614750574,
  0.15458257062638975

#### Simple CNN Regressor

In [8]:
age_model = Sequential()
age_model.add(Convolution2D(filters = 64, kernel_size = (3, 3), 
      padding = 'same',input_shape=(100,100,3), activation = 'relu'))
age_model.add(MaxPooling2D(pool_size= (2, 2)))
age_model.add(Convolution2D(filters = 32, kernel_size = (3, 3), 
                              padding = 'same', activation = 'relu'))
age_model.add(MaxPooling2D(pool_size= (2, 2)))
age_model.add(Dropout(0.5))
age_model.add(Flatten())
age_model.add(Dense(128, activation = 'relu'))
age_model.add(Dense(64, activation = 'relu'))
age_model.add(Dropout(0.5))
age_model.add(Dense(1, activation = 'linear'))

opt = keras.optimizers.Adam(learning_rate=0.0003)
age_model.compile(optimizer=opt, 
                  loss='mean_absolute_error', metrics=['mse'])

age_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 100, 100, 64)      1792      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 25, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)              

In [9]:
age_model.load_weights('history/age_estimator.h5')

In [10]:
with open('history/history_age.pickle', 'rb') as f:
    history_age = pickle.load(f)

In [12]:
history_age

{'loss': [5.011833998018718,
  4.784001982868827,
  4.685626285618316,
  4.606836673567995,
  4.532057291320913,
  4.463018444571333,
  4.4116422883988,
  4.362661898924125,
  4.319926655429083,
  4.2789706708526865,
  4.238263256956325,
  4.206879903667315,
  4.174175123413995,
  4.146666801530154,
  4.113106919047637,
  4.091955796897662,
  4.061452015336869,
  4.03501081998101,
  4.011095366672013,
  3.9857317527963527,
  3.959093846470826,
  3.9396166241479547,
  3.9189623424842126,
  3.8994001288561213,
  3.881787848082363,
  3.8540870415638864,
  3.834898856238983,
  3.819694605927157,
  3.797393575058203,
  3.7818421365305945,
  3.7627817718590886,
  3.7445694994291534,
  3.7289147742233704,
  3.704142393439347,
  3.694923012845378,
  3.678288522407093,
  3.6581441195563356,
  3.6426694034173432,
  3.6227472033581765,
  3.614435238157818,
  3.5998884285849613,
  3.5910563771566886,
  3.5784059470552516,
  3.5531502564131876,
  3.546941870389394,
  3.5290954492454287],
 'mse': [4